# Segmenting and Clustering Neighborhoods in Toronto :

## Create and prepare Toronto postal codes Dataframe

### Imports and Initialization

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
# Import dependencies

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Now assign the link of the website through which we are going to scrape the data and assign it to variable named website_url, and create a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping.

In [15]:
# getting data from aspecific source (wikipedia) 
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#Find the required table are
soup = BeautifulSoup(website_url,'lxml')

# extracting the raw table inside that webpage
table = soup.find('table', {'class': 'wikitable sortable'})

### Data processing 1 :  Extract the table to a base dataframe
The base dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [21]:
table_pr = table.find_all('tr')

# Extract the table header (Extract three columns PostalCode, Borough, and Neighborhood)
table_header_pr = table_pr[0].findAll('th')
table_header = [str(th).replace('<th>', '').replace('</th>', '').replace('\n', '') for th in table_header_pr]

# Extract the table body
table_body_pr = table_pr[1:]
table_body = [list(map(lambda td: str(td if not td.findAll("a") 
                                         else td.find("a").getText()).replace('<td>', '')
                          .replace('</td>', '').replace('\n', '')
                          , tr.findAll('td'))) for tr in table_body_pr]


# Create a base dataframe to clean and transform
df_base = pd.DataFrame(table_body, columns=table_header)

### Data processing 2 :  Ignore cells with a borough that is not assigned

In [22]:
# replace "Not assigned" to NaN : To use dropna function for dropping the cells that have an Not assigned borough
df_base.replace("Not assigned", np.nan, inplace=True)

# Drop cells with "Not assigned" borough 
df_base.dropna(subset=["Borough"], inplace=True)

### Data processing 3 :  Replace Not assigned neighborhood by the borough
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [23]:
df_base.Neighbourhood.fillna(df_base.Borough, inplace=True)

### Data processing 4 :  Combine neighborhood that have the same postal code into one row
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [24]:
# Join neighboorhood Strings that have the same postal code to one row 
df_base['Neighbourhood'] = df_base.groupby(['Postcode'])['Neighbourhood'].transform(lambda x: ', '.join(x))

# Drop duplicates
df_toronto = df_base.drop_duplicates()

## Visualize the appropriate pandas Dataframe

### 1. Visualizing the 10 first rows of the dataframe

In [25]:
df_toronto.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge, Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens, Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson, Garden District"


### 2. Print the number of rows of your dataframe

In [26]:
df_toronto.shape

(103, 3)

##### The number of rows in the final dataframe equal to : 103

In [44]:
print("Number of rows in the dataframe : %d" % df_toronto["Postcode"].shape[0])

Number of rows in the dataframe : 103
